In [1]:
%pylab inline
%load_ext autoreload
%autoreload 2

from jupyterthemes import jtplot
jtplot.style()

Populating the interactive namespace from numpy and matplotlib


In [2]:
import os, sys, time
import cPickle as pickle
sys.path.append('../EXP/')
import ClevelandMcGill as C
reload(C)

<module 'ClevelandMcGill' from '../EXP/ClevelandMcGill/__init__.pyc'>

In [3]:
N = 100000
images = np.zeros((N,100,100), dtype=np.bool)
labels = np.zeros((N), dtype=np.float32)

for n in range(N):
  
  sparse, image, label, parameters = C.Figure1.position_common_scale()
  
  images[n] = image
  labels[n] = label
  

In [4]:
X = images.astype(np.float32) - .5
y = (labels-np.min(labels))/(np.max(labels)-np.min(labels))
print 'memory usage', (X.nbytes + y.nbytes) / 1000000., 'MB'

memory usage 4000.4 MB


In [5]:
X3D = np.stack((X,)*3, -1)
print 'memory usage', (X3D.nbytes + y.nbytes) / 1000000., 'MB'

memory usage 12000.4 MB


In [6]:
import keras.applications

VGG19 = keras.applications.VGG19(include_top=False, weights='imagenet', input_shape=(100,100,3))

t0 = time.time()
features = VGG19.predict(X3D, verbose=True)
print 'VGG19 features done after', time.time()-t0

Using TensorFlow backend.


100000/100000 [==============================] - 276s 3ms/step
VGG19 features done after 275.932182789


In [7]:
oshape = features.shape
print 'features shape', oshape
print 'memory usage', (features.nbytes + y.nbytes) / 1000000., 'MB'

features shape (100000, 3, 3, 512)
memory usage 1843.6 MB


In [8]:
from keras import models
from keras import layers
from keras import optimizers
import keras.applications
from keras import backend as K
from keras.utils.np_utils import to_categorical

MLP = models.Sequential()

MLP.add(layers.Dense(256, activation='relu', input_dim=oshape[1]*oshape[2]*oshape[3]))
MLP.add(layers.Dropout(0.5))
MLP.add(layers.Dense(1, activation='linear')) # REGRESSION

sgd = optimizers.SGD(lr=0.0001, decay=1e-6, momentum=0.9, nesterov=True)
MLP.compile(loss='mean_squared_error', optimizer=sgd, metrics=['mse', 'mae']) # MSE for regression

In [9]:
t0 = time.time()
callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='auto')]

history = MLP.fit(features[0:60000].reshape(60000, oshape[1]*oshape[2]*oshape[3]), \
                  y[0:60000], \
                  epochs=1000, \
                  batch_size=32, \
                  validation_split=0.25,
                  callbacks=callbacks,
                  verbose=True)
print 'Fitting done', time.time()-t0

Train on 45000 samples, validate on 15000 samples
Epoch 1/1000
45000/45000 [==============================] - 7s 153us/step - loss: 0.0275 - mean_squared_error: 0.0275 - mean_absolute_error: 0.1036 - val_loss: 9.8928e-04 - val_mean_squared_error: 9.8928e-04 - val_mean_absolute_error: 0.0215
Epoch 2/1000
45000/45000 [==============================] - 7s 147us/step - loss: 0.0079 - mean_squared_error: 0.0079 - mean_absolute_error: 0.0670 - val_loss: 6.3442e-04 - val_mean_squared_error: 6.3442e-04 - val_mean_absolute_error: 0.0195
Epoch 3/1000
45000/45000 [==============================] - 7s 146us/step - loss: 0.0067 - mean_squared_error: 0.0067 - mean_absolute_error: 0.0612 - val_loss: 3.6496e-04 - val_mean_squared_error: 3.6496e-04 - val_mean_absolute_error: 0.0154
Epoch 4/1000
45000/45000 [==============================] - 7s 147us/step - loss: 0.0059 - mean_squared_error: 0.0059 - mean_absolute_error: 0.0574 - val_loss: 5.1544e-04 - val_mean_squared_error: 5.1544e-04 - val_mean_absol

45000/45000 [==============================] - 7s 148us/step - loss: 0.0024 - mean_squared_error: 0.0024 - mean_absolute_error: 0.0352 - val_loss: 7.0385e-05 - val_mean_squared_error: 7.0385e-05 - val_mean_absolute_error: 0.0063
Epoch 35/1000
45000/45000 [==============================] - 7s 148us/step - loss: 0.0023 - mean_squared_error: 0.0023 - mean_absolute_error: 0.0349 - val_loss: 1.2575e-04 - val_mean_squared_error: 1.2575e-04 - val_mean_absolute_error: 0.0097
Epoch 36/1000
45000/45000 [==============================] - 7s 146us/step - loss: 0.0024 - mean_squared_error: 0.0024 - mean_absolute_error: 0.0349 - val_loss: 1.1306e-04 - val_mean_squared_error: 1.1306e-04 - val_mean_absolute_error: 0.0092
Epoch 37/1000
45000/45000 [==============================] - 7s 147us/step - loss: 0.0024 - mean_squared_error: 0.0024 - mean_absolute_error: 0.0350 - val_loss: 1.0607e-04 - val_mean_squared_error: 1.0607e-04 - val_mean_absolute_error: 0.0090
Epoch 38/1000
45000/45000 [===============

In [10]:
y_pred = MLP.predict(features[60000:].reshape(40000, oshape[1]*oshape[2]*oshape[3]))

In [11]:
import sklearn.metrics
y_test = y[60000:]
np.log2(sklearn.metrics.mean_absolute_error(y_pred*100, y_test*100)+.125)

0.33803462683127294